<a href="https://colab.research.google.com/github/benardt/ML/blob/main/oring_optim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import shutil
shutil.unpack_archive("/content/drive/My Drive/data/test.zip", "/tmp")

Mounted at /content/drive


In [3]:
!pip install thop 1>/dev/null
!pip install optuna 1>/dev/null

import os
import time
import datetime
from PIL import Image, ImageOps

from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np

import optuna
from optuna.trial import TrialState

import thop
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, utils



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
!pip install livelossplot --quiet
from livelossplot import PlotLosses
!pip install import-ipynb
import import_ipynb
from importlib import reload # reload 

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=436dd6ef2e6039cfe4b6b2b3883c702682ceffaa7e183cfa30238af13f889027
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [5]:
!rm -r './mylib.ipynb'
!cp '/content/drive/My Drive/Colab Notebooks/mylib.ipynb' .
import mylib as ml
reload(ml)
ml.mytest('This is a test.')

rm: cannot remove './mylib.ipynb': No such file or directory
importing Jupyter notebook from mylib.ipynb
importing Jupyter notebook from mylib.ipynb
test import... This is a test.


In [18]:
FOLDER = 'oring11'

OUTPUT_SIZE = 63
BATCH_SIZE = 32
EPOCHS = 5

shutil.unpack_archive('/content/drive/My Drive/data/'+FOLDER+'.zip', '/tmp')

def get_data(isprint=False):

    trans_in = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                          transforms.ToTensor()])
    
    interpol = transforms.InterpolationMode.NEAREST
    trans_out = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                          transforms.Resize(OUTPUT_SIZE,interpolation=interpol),
                          transforms.ToTensor()])   

    class OringLandmarksDataset(Dataset):
        """Landmarks dataset."""

        def __init__(self, root_dir):
            """
            Args:
                root_dir (string): Directory with all the images.
            """

            self.root_dir = root_dir
            self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(root_dir, "Masks"))))

        def __len__(self):
            return len(self.imgs)

        def transform(self, x, y):

            image = trans_in(x)
            mask = trans_out(y)
            return image, mask

        def __getitem__(self, idx):
            if torch.is_tensor(idx):
                idx = idx.tolist()

            img_name = os.path.join(self.root_dir, "PNGImages", self.imgs[idx])
            im_invert = Image.open(img_name).convert('L')
            image = ImageOps.invert(im_invert)

            path = self.imgs[idx]

            mask_name = os.path.join(self.root_dir, "Masks", self.masks[idx])
            mask_invert = Image.open(mask_name).convert('L')
            mask = ImageOps.invert(mask_invert)

            x, y = self.transform(image, mask)
            return x, y

    trans_dataset = OringLandmarksDataset(root_dir='/tmp/'+FOLDER)

    train_len = int(0.6*len(trans_dataset))
    valid_len = len(trans_dataset)-train_len
    TrainData, ValidData = torch.utils.data.random_split(trans_dataset,[train_len, valid_len])

    mydataloader = { 'train':[],'valid':[]}
    mydataloader['train'] = DataLoader(TrainData, batch_size=BATCH_SIZE,
                            shuffle=True, pin_memory=True,num_workers=2)
    mydataloader['valid'] = DataLoader(ValidData, batch_size=BATCH_SIZE,
                            shuffle=True, pin_memory=True,num_workers=2)

    if isprint:
      for i_batch, (x,y) in enumerate(mydataloader['train']):
        print(i_batch, x[0].size(), y[0].size())
        x = transforms.functional.resize(x, OUTPUT_SIZE)
        ml.images_show(x[0],y[0],5)
        if i_batch == 2:
          break

    return mydataloader

dataloader = get_data(isprint=False)


In [38]:
def define_model(trial):

  layers = []
  out_features = [None,None]
  kernels_size = [None,None]

  i = 1 # first layer
  out_features[i] = trial.suggest_int("n_channels{}".format(i), 32, 128, 32)
  kernels_size[i] = trial.suggest_int("ker_sizes{}".format(i), 7, 23, 4)

  # 1st layer
  layers.append(nn.Conv2d(1,out_features[1],kernels_size[1],1,int((kernels_size[1]-1)/2)))
  layers.append(nn.MaxPool2d(15,2,7))

  # 2nd layer
  layers.append(nn.Conv2d(out_features[1],128,5,1,2))
  layers.append(nn.MaxPool2d(5,2,2))

  # 3rd layer
  layers.append(nn.Conv2d(128,128,3,1,1))
  layers.append(nn.MaxPool2d(3,2,1))

  # last layer
  layers.append(nn.Conv2d(128,1,1,1,0))

  return nn.Sequential(*layers)


In [39]:
def objective(trial):
    accuracy = ml.DiceLoss()
    liveloss = PlotLosses()
    model = define_model(trial).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 0.001)

    #past_date = datetime.datetime.now().replace(microsecond=0)

    all_logs = []
    for epoch in range(EPOCHS):
        logs = {}
        for phase in ['train', 'valid']:

            running_loss = 0.0
            running_corrects = 0

            for x, y in iter(dataloader[phase]):
                x,y = x.to(device),y.to(device)

                if phase == 'train':
                    model.train()
                    yhat = model(x)
                    loss = criterion(yhat, y)
                    for param in model.parameters():
                        param.grad = None
                    loss.backward()
                    optimizer.step()

                else:
                    model.eval()
                    with torch.no_grad():
                        yhat = model(x)
                        loss = criterion(yhat, y)

                # loss
                running_loss += loss.detach() * x.detach().size(0)
                # accuracy
                preds = (yhat.detach() > 0.0)*1
                acc = 100*(1 - accuracy(preds, y.detach()))
                running_corrects += acc * x.detach().size(0)


            epoch_loss = running_loss.item() / len(dataloader[phase].dataset)
            epoch_acc = running_corrects.item() / len(dataloader[phase].dataset)

            prefix = ''
            if phase == 'valid':
                prefix = 'val_'

            logs[prefix + 'log loss'] = epoch_loss
            logs[prefix + 'accuracy'] = epoch_acc

        all_logs.append(logs)
        #liveloss.update(logs)
        #liveloss.send()

        trial.report(epoch_acc, epoch)
        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    # future_date = datetime.datetime.now().replace(microsecond=0)
    # difference = (future_date - past_date)
    # total_seconds = int(difference.total_seconds())

    flops, _params = thop.profile(model, inputs=(torch.randn(1, 1, 500, 500).to(device),), verbose=False)
    return epoch_acc


In [41]:
torch.cuda.empty_cache()

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


[I 2021-07-07 20:26:06,527] A new study created in memory with name: no-name-291aeb8b-1dca-4109-a802-9aadf56917e2
[I 2021-07-07 20:29:26,581] Trial 0 finished with value: 48.687142457916885 and parameters: {'n_channels1': 64, 'ker_sizes1': 7}. Best is trial 0 with value: 48.687142457916885.
[I 2021-07-07 20:34:01,459] Trial 1 finished with value: 74.84446508416623 and parameters: {'n_channels1': 96, 'ker_sizes1': 7}. Best is trial 1 with value: 74.84446508416623.
[I 2021-07-07 20:36:06,904] Trial 2 finished with value: 73.63445226196738 and parameters: {'n_channels1': 32, 'ker_sizes1': 7}. Best is trial 1 with value: 74.84446508416623.
[I 2021-07-07 20:41:03,764] Trial 3 finished with value: 90.8334840215676 and parameters: {'n_channels1': 64, 'ker_sizes1': 23}. Best is trial 3 with value: 90.8334840215676.
[I 2021-07-07 20:44:23,871] Trial 4 finished with value: 74.95924842188322 and parameters: {'n_channels1': 64, 'ker_sizes1': 7}. Best is trial 3 with value: 90.8334840215676.


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  90.8334840215676
  Params: 
    n_channels1: 64
    ker_sizes1: 23
